In [30]:
import os
import sys
import pandas as pd
import pprint
import psycopg2

In [36]:
# connect postgreSQL
sys.path.append('..')
import connection_info
users = connection_info.users
dbnames = connection_info.dbnames
passwords = connection_info.passwords
conn = psycopg2.connect(" user=" + users +" dbname=" + dbnames +" password=" + passwords)

In [9]:
# excexute sql
cur = conn.cursor()
cur.execute('SELECT * FROM work.reserve_tb;')
results = cur.fetchall()

In [15]:
type(results)

list

In [17]:
reserve = pd.read_sql(sql="SELECT * FROM work.reserve_tb;", con=conn, index_col='reserve_id' )

In [18]:
reserve.shape

(4030, 8)

## 2-2

In [7]:
reserve2_2 = pd.read_sql(
sql=
    """
SELECT *
FROM work.reserve_tb

-- インデックスを効かせるために、checkin_dateでも絞り込み
WHERE checkin_date BETWEEN '2016-10-10' AND '2016-10-13'
  AND checkout_date BETWEEN '2016-10-13' AND '2016-10-14'
""",
con=conn, index_col='reserve_id' )    

In [8]:
reserve2_2

,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
reserve_id,,,,,,,,
r285,h_121,c_67,2016-09-27 06:13:19,2016-10-12,12:00:00,2016-10-14,4,184000
r514,h_74,c_120,2016-10-06 03:12:04,2016-10-11,12:30:00,2016-10-14,2,28800
r1066,h_205,c_261,2016-09-14 02:57:59,2016-10-11,10:00:00,2016-10-14,4,85200
r1481,h_116,c_364,2016-09-17 17:45:39,2016-10-11,11:30:00,2016-10-13,4,107200
r1547,h_149,c_377,2016-09-27 08:19:24,2016-10-10,11:00:00,2016-10-13,2,153600
r1710,h_59,c_422,2016-09-19 04:17:25,2016-10-10,12:00:00,2016-10-13,2,148800
r1933,h_113,c_477,2016-09-24 09:04:26,2016-10-12,11:30:00,2016-10-13,4,77200
r2059,h_9,c_517,2016-09-19 15:32:35,2016-10-11,12:30:00,2016-10-13,3,188400
r2116,h_77,c_527,2016-10-05 00:44:09,2016-10-11,09:00:00,2016-10-13,4,353600


## 2-3 データ値に基づかないサンプリング

Not Awesome

In [19]:
reserve2_3NA = pd.read_sql(
sql=
    """
SELECT *
FROM work.reserve_tb

-- データ行ごとに乱数を生成し、乱数の小さい順にデータを並び替え
ORDER BY RANDOM()

-- サンプリングする件数をLIMIT句で指定
-- 事前にカウントしたデータ数を入力し、抽出する割合をかけ、ROUNDによって四捨五入
LIMIT ROUND(4030 * 0.5)
""",
con=conn, index_col='reserve_id' )    

In [20]:
reserve2_3NA.shape

(2015, 8)

Awesome

In [15]:
reserve2_3Awe = pd.read_sql(
sql=
"""
SELECT *
FROM work.reserve_tb

-- 行ごとに乱数を生成し、0.5以下のデータ行のみ絞り込み
WHERE RANDOM() <= 0.5
""",
con=conn, index_col='reserve_id' )    

In [16]:
reserve2_3Awe.shape

(2034, 8)

## 2-3 データ値に基づかないサンプリング

Awesome

In [26]:
reserve2_4Awe = pd.read_sql(
sql=
"""
-- WITH句によって、一時テーブルreserve_tb_randomを生成
WITH reserve_tb_random AS(
  SELECT
    *,

    -- customer_idに対して一意の値となる乱数の生成
    -- 生成した乱数をcustomer_idごとにまとめて、1番目の値を取り出す
    FIRST_VALUE(RANDOM()) OVER (PARTITION BY customer_id) AS random_num

  FROM work.reserve_tb
)
-- *ですべての列を抽出しているが、random_numを外したい場合は列を指定する必要あり
SELECT *
FROM reserve_tb_random

-- 50%サンプリング、customer_idごとに設定された乱数が0.5以下の場合に抽出
WHERE random_num <= 0.5
""",
con=conn, index_col='reserve_id' )
reserve2_4Awe

,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,random_num
reserve_id,,,,,,,,,
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,0.135888
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,0.161870
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.401646
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.464762
r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,0.309984
r64,h_112,c_10,2018-03-07 00:43:22,2018-03-13,12:30:00,2018-03-14,3,117000,0.149337
r63,h_142,c_10,2017-09-11 19:41:28,2017-09-13,10:00:00,2017-09-15,2,37200,0.172422
r62,h_21,c_10,2017-07-29 17:48:41,2017-08-06,12:00:00,2017-08-08,4,89600,0.012070
r61,h_62,c_10,2017-03-12 00:45:50,2017-04-08,10:00:00,2017-04-10,2,187200,0.205620


In [21]:
reserve2_4Awe_ex = pd.read_sql(
sql=
"""
  SELECT
    *,

    -- customer_idに対して一意の値となる乱数の生成
    -- 生成した乱数をcustomer_idごとにまとめて、1番目の値を取り出す
    FIRST_VALUE(RANDOM()) OVER (PARTITION BY customer_id) AS random_num

  FROM work.reserve_tb

""",
con=conn, index_col='reserve_id' )   

In [27]:
reserve2_4Awe_ex.loc[reserve2_4Awe_ex['customer_id']=='c_1']

,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,random_num
reserve_id,,,,,,,,,
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,0.896331
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,0.991807
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.668622
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,0.004954
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.526296
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,0.997575
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.481073
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.126287
